In [1]:
%pip install langchain==0.0.319
%pip install openai==0.28.1
%pip install chromadb==0.4.14
%pip install sentence-transformers
%pip install pypdf pdfplumber
%restart_python

  Using cached langchain-0.0.319-py3-none-any.whl.metadata (15 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.319-py3-none-any.whl (1.9 MB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.16
    Uninstalling langchain-0.1.16:90m╺━━━━━━━━━━━━━━━━━━━ 1/2 [langchain]
      Successfully uninstalled langchain-0.1.16━━━━━━━━━━━━━━━━━━━ 1/2 [langchain]
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [langchain]  WARNING: The scripts langchain and langchain-server are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or,

UsageError: Line magic function `%restart_python` not found.


In [2]:
import os
import shutil
import hashlib
import json
import pdfplumber
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Configuration
pdf_files = [
    "HKJC_Annual Report 2425 part1.pdf",
    "HKJC_Annual Report 2425 part2.pdf",
    "Jockey202425.pdf",
    "HKJC_140A.pdf"
]
persist_dir = "./chroma_db_1223_8"

# Helper
def extract_year_values_from_line(line):
    """Extracts two year values from a line (e.g., 2025 and 2024)."""
    values = [
        v.replace(",", "").strip()
        for v in line.split()
        if v.replace(",", "").strip().replace(",", "").isdigit()
    ]
    return values[:2] if len(values) >= 2 else None

# Splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

all_chunks = []
seen_hashes = set()
table_count = 0

for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    for doc in docs:
        doc.metadata.update({
            "source": pdf_path,
            "fallback_prompt": "You are a PDF assistant. If data is unclear or missing, say 'Not enough information.'",
            "type": "text"
        })

    # Deduplicate text chunks
    text_chunks = splitter.split_documents(docs)
    for chunk in text_chunks:
        content_hash = hashlib.md5(chunk.page_content.strip().encode()).hexdigest()
        if content_hash not in seen_hashes:
            all_chunks.append(chunk)
            seen_hashes.add(content_hash)

    # Table extraction using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        prev_table, prev_header, prev_page, prev_start_page = None, None, None, None

        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            text = page.extract_text() or ""
            lines = text.splitlines()

            for line in lines:
                # Dynamically match lines that contain a label followed by two numbers (e.g., 2025 and 2024 values)
                if any(keyword in line.lower() for keyword in ["sale", "purchase", "deposit", "exchange", "interest", "cash", "equivalent", "asset", "equipment"]):
                    extracted = extract_year_values_from_line(line)
                    if extracted:
                        try:
                            value_2025, value_2024 = extracted
                            label = line.split(str(value_2025))[0].strip()

                            table_count += 1
                            fallback_json = {
                                label: {
                                    "2025": int(value_2025),
                                    "2024": int(value_2024)
                                }
                            }

                            table_doc = Document(
                                page_content=(
                                    f"{label} (from fallback):\n"
                                    f"2025: {value_2025} million HKD\n"
                                    f"2024: {value_2024} million HKD"
                                ),
                                metadata={
                                    "source": pdf_path,
                                    "type": "table",
                                    "table_id": f"{pdf_path}_fallback_row_{table_count}",
                                    "columns": "Year, Value (HK$ million)",
                                    "page_start": i + 1,
                                    "page_end": i + 1,
                                    "json": json.dumps(fallback_json),
                                    "fallback_prompt": f"Patch: This is a fallback row from `{label}` in the cash flow statement or similar. Use for financial analysis."
                                }
                            )
                            all_chunks.append(table_doc)
                        except Exception as e:
                            print(f"⚠️ Error on page {i+1} for line: {line} — {e}")

            # Add full page text if no tables were found
            if not tables:
                all_chunks.append(Document(
                    page_content=text,
                    metadata={
                        "source": pdf_path,
                        "type": "financial_statement",
                        "page": i + 1,
                        "fallback_prompt": "This page may contain narrative or financial data. Extract manually if needed."
                    }
                ))
                continue

            # Process tables normally
            for table in tables:
                clean_table = [
                    [cell.strip() if cell else "" for cell in row]
                    for row in table if any(cell for cell in row)
                ]
                if not clean_table or len(clean_table) < 2:
                    continue

                header, content = clean_table[0], clean_table[1:]

                if prev_table:
                    table_count += 1
                    table_str = "\n".join(["\t".join(row) for row in [prev_header] + prev_table])
                    table_doc = Document(
                        page_content=table_str,
                        metadata={
                            "source": pdf_path,
                            "type": "table",
                            "table_id": f"{pdf_path}_table_{table_count}",
                            "columns": ", ".join(prev_header),
                            "page_start": prev_start_page,
                            "page_end": prev_page,
                            "fallback_prompt": "This is a financial table. Look for metrics like cash flows, ROE, or liquidity ratios."
                        }
                    )
                    all_chunks.append(table_doc)

                prev_table = content
                prev_header = header
                prev_start_page = i + 1
                prev_page = i + 1

        # Final flush
        if prev_table:
            table_count += 1
            table_str = "\n".join(["\t".join(row) for row in [prev_header] + prev_table])
            table_doc = Document(
                page_content=table_str,
                metadata={
                    "source": pdf_path,
                    "type": "table",
                    "table_id": f"{pdf_path}_table_{table_count}",
                    "columns": ", ".join(prev_header),
                    "page_start": prev_start_page,
                    "page_end": prev_page,
                    "fallback_prompt": "This is a financial table. Review for key indicators."
                }
            )
            all_chunks.append(table_doc)

# Rebuild vector DB
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

if not all_chunks:
    print("❌ No chunks extracted.")
else:
    embedding = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )

    vectordb = Chroma.from_documents(
        documents=all_chunks,
        embedding=embedding,
        persist_directory=persist_dir
    )
    vectordb.persist()

    # Report
    text_chunks = [c for c in all_chunks if c.metadata.get("type") == "text"]
    table_chunks = [c for c in all_chunks if c.metadata.get("type") == "table"]

    print("\n✅ Vector store created successfully.")
    print(f"📁 Saved to: {persist_dir}")
    print(f"📄 Total text chunks: {len(text_chunks)}")
    print(f"📊 Total table chunks: {len(table_chunks)}")
    print(f"🔎 Total fallback entries: {table_count}")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



✅ Vector store created successfully.
📁 Saved to: ./chroma_db_1223_8
📄 Total text chunks: 930
📊 Total table chunks: 151
🔎 Total fallback entries: 151


In [ ]:
%pip install --upgrade langchain==0.1.16 langchain-community==0.0.33 langchain-openai==0.0.8


In [ ]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("HKJC_140A.pdf")
docs = loader.load()
print(f"📄 Loaded {len(docs)} pages")
print("📝 Sample text:", docs[0].page_content[:300])
# ✅ Vectorstore path (same as you used)
persist_dir = "./chroma_db_1222_5"

# ✅ Load the same embedding model used to build the DB
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# ✅ Load the vectorstore from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# ✅ Check how many documents are in the store
print("📄 Number of documents in vectorstore:", vectordb._collection.count())

# ✅ Ask a test question
user_query = "who are An An and Ke Ke?"
results = vectordb.similarity_search(user_query, k=3)

# ✅ Print results
if results:
    for i, doc in enumerate(results, 1):
        print(f"\n🔹 Result {i}:\n{doc.page_content[:500]}")  # truncate output
else:
    print("❌ No relevant documents found.")

for i, doc in enumerate(docs[:3]):
    print(f"\nPage {i+1} content length: {len(doc.page_content)}")
    print(doc.page_content[:300])

📄 Loaded 118 pages
📝 Sample text: 


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


📄 Number of documents in vectorstore: 1001

🔹 Result 1:
3
Board of Stewards
董事局
Mr Michael T H Lee 
JP 
(Chairman)
利子厚先生(主席)
The Hon Martin Liao 
GBM GBS JP 
(Deputy Chairman)
廖長江先生(副主席)
Dr Silas S S Yang 
JP
楊紹信博士
Miss Anita Fung Yuen Mei 
BBS JP
馮婉眉小姐
Mr Philip K W Lo
羅啟華先生
Mr Andrew W B R Weir 
BBS MBE JP
韋安祖先生
Mr Lester G Huang 
SBS JP
黃嘉純先生
Dr Henry H L Chan
陳衍里醫生
The Hon Bernard Charnwut Chan 
GBM GBS JP
陳智思先生
Mr Jackson Woo Ka Biu
胡家驃先生
Mrs Ann Kung Yeung Yun Chi 
BBS JP
龔楊恩慈女士
Mr Nicholas D Hunsworth
孔思和先生

🔹 Result 2:

SECTOR

🔹 Result 3:
	BEAUTY JOY
美麗同享
A S Cruz
告東尼
Eleanor Kwok Law Kwai
Chun & Patrick Kwok Ho
Chuen
郭羅桂珍與郭浩泉	

Page 1 content length: 0


Page 2 content length: 0


Page 3 content length: 0

